In [ ]:
from mpl_toolkits.basemap import Basemap, cm
from netCDF4 import Dataset, date2index
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import pandas as pd
import numpy as np
from pylab import *

# Reading netCDF File

- Here we read a netCDF file using an OpenDAP link.
- We find the nearest latitude and longitude available to the point of focus, by calculating and sorting the squared difference values from the point of consideration and then selecting the required initial values. 
- Here we print and calculate these values for the next step

**Instructions:**
- 1) Enter the OpenDAP link of the dataset in the following script.
- 2) Fill in the latitude and longitude values of the point of consideration
- 3) Enter the number of initial values needed from the sorted array
      - for example : In a 1 deg resolution dataset for extracting data of 1 deg box, we need to find nearest 2 values for both latitude and longitude 

In [ ]:
# Reading in the netCDF file
data = Dataset('http://apdrc.soest.hawaii.edu:80/dods/public_data/NOAA_SST/OISST_AVHRR/daily_v2.0')

# Storing the lat and lon data into the variables 
lat = data.variables['lat'][:]
lon = data.variables['lon'][:]

# Storing the lat and lon of BOB
lat_katmandu = 14.094
lon_katmandu = 88.268

# Squared difference of lat and lon 
sq_diff_lon = []
sq_diff_lon.append((lon - lon_katmandu)**2)
sq_diff_lat = []
sq_diff_lat.append((lat - lat_katmandu)**2)


# Identifying the index of the minimum value for lat and lon 
min_index_lon = sorted((sq_diff_lon)[0])[0:5]
min_index_lat = sorted((sq_diff_lat)[0])[0:5]
z = [] # for Longitude
x = [] # for latitude

for i in range(len(min_index_lon)):
    z.append(min_index_lon[i])

for i in range(len(min_index_lat)):
    x.append(min_index_lat[i])

print(z,x)

# Calculating Index of the Identified Lat and Lon

Here the indexes of the latitude and longitude nearest to the point of focus are calculated, which will be used further in next step to extrapolate the data.



In [ ]:
# Identifying the index of the minimum value for lat and lon 

f=[]# for long.
r=[]# for latt.

for i,j in enumerate(sq_diff_lon):
    for g in range(len(min_index_lon)):
        for k,l in enumerate(j):
     
               if l==z[g]:
                  f.append(k)

for i,j in enumerate(sq_diff_lat):
    for g in range(len(min_index_lat)):
        for k,l in enumerate(j):
     
               if l==x[g]:
                  r.append(k)
#f = f[0:2]
#r = r[0:2]
print(f,r)

# Calculating Monthly Average of the Data over the Region

- 1) Here we calculate the average of the data over the points of focus.
- 2) First we generate all the possible coordinates using the indexes we generated
- 3) Then we extract the data pointwise in a list 

**Instructions:**

For Part 1 : (When resolution is of 1 day)
- 1) Extract the desired variable data needed.
- 2) For break statement choose the total number of days

For Part 2 : (When resolution is of 1 month)
- 1) Extract the desired variable data needed.
- 2) For break statement choose the total number of months


# Part 1 (When Resolution is of 1 day)

In [ ]:
sst = data.variables['sst']
def avg():
    
    possible_coordinates=[]

    for a in range(len(f)):
        for b in range(len(r)):
            k = [f[a],r[b]]
            possible_coordinates.append(k)
            
    month  = [30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,
              30,31,30,31,31,29,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31]
    
    point_month_wise = []
    
    for points in possible_coordinates:
        lat = points[1];lon = points[0]
        start = 0;average_month = []
        
        for count in range(1,13):
            ind = count%48
            days = month[ind-1]
            sum_for_month = 0
            for i in range(start,start+days+1):
                val = sst[i,lat,lon]
                sum_for_month += val
            start += days
            
            if start > 14092:
                break
            
            avg = sum_for_month/days
            average_month.append(avg)
        point_month_wise.append(average_month)
    return point_month_wise

m = avg()          

# Part 2 (When Resolution is of 1 month)

In [ ]:
def avg():
    
    possible_coordinates=[]

    for a in range(len(f)):
        for b in range(len(r)):
            k = [f[a],r[b]]
            possible_coordinates.append(k)
            
    point_month_wise = []
    
    for points in possible_coordinates:
        lat = points[1];lon = points[0]
        month = []

        for count in range(498):
            val = sst[count,0,lat,lon]
            month.append(val)    
                       
        point_month_wise.append(month)
    return point_month_wise
m = avg()         

# Average Over the Points

Here we average the data for all the points 

In [ ]:
final_avg_for_all_months = []

for i in range(len(m[0])):
    summ = 0
    for j in range(len(m)):
        summ += m[j][i]
    avg_for_all_pts_for_a_month = summ/(len(m))
    final_avg_for_all_months.append(avg_for_all_pts_for_a_month)
print(len(final_avg_for_all_months))

print(final_avg_for_all_months)

# Saving Data and Generating csv File

- 1) Here we first generate a data frame for the total number periods available.
- 2) Then we update it with data updated in an array created before that contains the averaged monthly data.

**Instructions:**

- 1) Set the starting date and periods according to the data set available
- 2) Set the name of the csv file accordingly 

In [ ]:
date_range = pd.date_range(start='1979/01/01', periods=498, freq='M')

df = pd.DataFrame(0, columns = ['sst'], index = date_range)

dt = np.arange(0,498)

for time_index in dt:
    df.iloc[time_index] = final_avg_for_all_months[time_index]
    
df.to_csv('SST_Monthly.csv')

# Sorting the Data

Here we try to segregate the data needed accordingly and generate a new csv file for it.

**Instructions:**

- 1) Decide the month for which the data is needed. (for example: for February [12*i + 1]).
- 2) Generate the data frame for the number of data points available.
- 3) Set the name and save the csv file.



In [ ]:
data = pd.read_csv('SST_Monthly.csv')

lis = []

for i in range(0,42):
    lis.append(data.Temparature[12*i])

dt = np.arange(1982,2021)
df = pd.DataFrame(0, columns = ['Temparature'], index = dt)

for t in range(len(lis)):
    df.iloc[t] = lis[t]
    
df.to_csv('JanuarySST_Monthly.csv')

# Plotting the Data 

Here we try to plot our data in the two possible ways i.e. a colour plot and a line chart 

- 1) Part 1 : (Colour Plot)
     - Enter the OpenDAP link for dataset.
     - Enter the period for which you need to visualize the data.
     - Enter the latitude and longitude bounds for which the data is needed to be visualized.
- 2) Part 2 : (Line Chart along with Linear Regression)
     - Enter the csv file for whose data visualization is needed.
     - Extract the data in the variables needed and plot according to the depended and indepedent variables.

# Part 1 (Colour Plot)

In [ ]:
date = datetime(2003,1,1,0)
# open dataset.
dataset = Dataset('http://apdrc.soest.hawaii.edu:80/dods/public_data/satellite_product/MODIS_Aqua/4km_day')
timevar = dataset.variables['time']
timeindex = date2index(date,timevar)
# read u current
uc = dataset.variables['sst'][timeindex,:].squeeze()
lats = dataset.variables['lat'][:]
lons = dataset.variables['lon'][:]
lons, lats = np.meshgrid(lons,lats)
# create figure, axes instances.
fig = plt.figure(figsize=(12,9))
ax = fig.add_axes([0.05,0.05,0.9,0.9])
# create Basemap instance.
m = Basemap(projection='mill',llcrnrlat=5.734,urcrnrlat=24.3777,llcrnrlon=78.8982,urcrnrlon=95.0488, resolution="c")
m.drawcoastlines()
clevs = [0,1,2.5,5,7.5,10,15,20,30,40,50,70,100,150,200,250,300,400,500,600,750]
im1 = m.pcolormesh(lons,lats,uc,shading='flat',cmap=plt.cm.jet,latlon=True)
cb = m.colorbar(im1,"bottom", size="5%", pad="1%")
ax.set_title('SST for 2003-01-01')
plt.show()

# Part 2 (Line Chart along with Linear regression for Time Series data)

In [ ]:
data = pd.read_csv('SST_Monthly.csv')
z = np.mean(data.Temparature)
k = data.Temparature-z

slope, intercept, r_value, p_value, std_err = stats.linregress(data.Date, k)
print(slope)

def predict(x):
    return slope * x + intercept

fitLine = predict(data.Date)
plt.figure(figsize=(25,10))
ax = plt.axes()
ax.xaxis.set_major_locator(plt.MaxNLocator(16))
plt.xticks(rotation=45, fontsize='20')
plt.yticks(fontsize='20')
plt.title('SST Anomaly on May', fontsize='30')
plt.xlabel("Time", fontsize='25')
plt.ylabel("Anomaly", fontsize='25')
plt.plot(data.Date, k)
plt.figtext(.6, .8, "Slope = 0.0163 ",fontsize = '25')
plt.plot(data.Date, fitLine, c='r')
plt.show()

# Part 2 (Line Chart along with Linear regression for the Sorted data)

In [ ]:
data = pd.read_csv('JanuarySST_Monthly.csv')
z = np.mean(data.Temparature)
k = data.Temparature-z

slope, intercept, r_value, p_value, std_err = stats.linregress(data.Date, k)
print(slope)

def predict(x):
    return slope * x + intercept

fitLine = predict(data.Date)
plt.figure(figsize=(25,10))
ax = plt.axes()
ax.xaxis.set_major_locator(plt.MaxNLocator(16))
plt.xticks(rotation=45, fontsize='20')
plt.yticks(fontsize='20')
plt.title('SST Anomaly on May', fontsize='30')
plt.xlabel("Time", fontsize='25')
plt.ylabel("Anomaly", fontsize='25')
plt.plot(data.Date, k)
plt.figtext(.6, .8, "Slope = 0.0163 ",fontsize = '25')
plt.plot(data.Date, fitLine, c='r')
plt.show()